This is the first notebook that I used to learn about plotly choropleth. I also put several other features (and data set) which help me to understand how to use this package.

In [1]:
import numpy as np
import pandas as pd


# https://plotly.com/python/getting-started/
# conda install -c plotly plotly=4.6.0
# conda install "notebook>=5.3" "ipywidgets>=7.2"
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from plotly.subplots import make_subplots
# import plotly.express as px
# import datetime

# 1. px.scatter_geo

In [ ]:
#included data from plotly about life expectation with iso for orthographic projection excercise 
import plotly.express as px
df = px.data.gapminder()

df.head()

In [ ]:
fig = px.scatter_geo(df.query("year==2007"), locations="iso_alpha", color="continent", projection="orthographic", size="pop")
fig.show()

# 2. go.Scattergeo

In [2]:
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
# country_df

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
0,Australia,2020-04-14 12:21:03,-25.000000,133.000000,6494,61,2186,4247,25.506978,NaN,NaN,0.939329,36,AUS
1,Austria,2020-04-14 12:13:01,47.516200,14.550100,14146,384,7633,6129,157.066086,NaN,NaN,2.714548,40,AUT
2,Canada,2020-04-14 12:20:47,60.001000,-95.001000,25680,780,7758,17142,67.836544,NaN,NaN,3.037383,124,CAN
3,China,2020-04-14 10:39:48,30.592800,114.305500,83306,3345,78195,1766,5.930619,NaN,NaN,4.015317,156,CHN
4,Denmark,2020-04-14 12:13:01,56.000000,10.000000,6691,285,2409,3997,115.517360,NaN,NaN,4.259453,208,DNK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,West Bank and Gaza,2020-04-14 12:13:01,31.952200,35.233200,308,2,62,244,6.037539,NaN,NaN,0.649351,275,PSE
181,Western Sahara,2020-04-14 12:13:01,24.215500,-12.885800,6,0,0,6,1.004470,NaN,NaN,0.000000,732,ESH
182,Yemen,2020-04-14 12:13:01,15.552727,48.516388,1,0,0,1,0.003353,NaN,NaN,0.000000,887,YEM
183,Zambia,2020-04-14 12:13:01,-13.133897,27.849332,45,2,30,13,0.244779,NaN,NaN,4.444444,894,ZMB


In [3]:
country_df.columns = map(str.lower, country_df.columns)
country_df = country_df.rename(columns={'country_region': 'country', 'long_': 'long', 'iso3':'iso'})

In [4]:
# convert the data to log scale
# np.seterr(divide = 'ignore') #numpy.seterr(divide = 'warn')
country_df['confirmed_log'] = np.log(country_df['confirmed'])
# country_df.head()

In [6]:
# color scale for the plot, otherwise the default "plasma" color scale will appear
# I prefer the plasma scale though 
scl = [0,"rgb(0, 0, 200)"],[0.125,"rgb(0, 25, 255)"],\
[0.25,"rgb(0, 152, 255)"],[0.375,"rgb(44, 255, 150)"],\
[0.5,"rgb(151, 255, 0)"],[0.625,"rgb(255, 234, 0)"],[0.75,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]

In [12]:
# Creating new dict for label dataframe 
marker_text = {'Country': country_df['country'],
        'Confirmed': country_df['confirmed'],
        'Death': country_df['deaths'],
         'Recovered' : country_df['recovered']
        }

marker_text_df = pd.DataFrame (marker_text, columns = ['Country','Confirmed','Death','Recovered'])
# marker_text_df.head()

# removing .0 in str format
for col in marker_text_df.columns:
    marker_text_df[col] = marker_text_df[col].astype(str).replace('\.0', '', regex=True)

# creating new column in label_df (called 'text') with the template for the text in choropleth 
marker_text_df['text'] = '<b>' + marker_text_df['Country'] + '</b>' + '<br>' + \
    'Confirmed : ' + marker_text_df['Confirmed'].astype(str) + '<br>' + \
    'Death : ' + marker_text_df['Death'].astype(str) + '<br>' + \
    'Recovered : ' + marker_text_df['Recovered'].astype(str)

In [20]:
# using scattergeo feature in plotly with corona data
# can be improved by putting the deaths, recovered and confirmed cases in the hovertext

fig = go.Figure(data=go.Scattergeo(
        lon = country_df['long'],
        lat = country_df['lat'],
        text = marker_text_df['text'],
        locations = country_df['iso'],
        hovertemplate=
        "%{text}<br>"
        "<extra><b>%{location}</b></extra>",
        mode = 'markers',
        marker_colorscale = scl,
        marker_cmin = 0,
        marker_color = country_df['confirmed_log'],
        marker_cmax = max(country_df['confirmed_log']),
        marker_size = country_df['confirmed_log']*3,
        marker_colorbar=dict(
            title="Number of Cases",
#             tick0= 0,
            tickvals=np.linspace(0,np.log(country_df['confirmed'].max()),7).astype(int),
            ticktext=np.append(0, np.exp(np.round(np.linspace(2,np.log(country_df['confirmed'].max()),6))).astype(int)),
        ),
    ))

fig.update_geos(projection_type="orthographic")
fig.update_layout(title = 'Confirmed Cases Globally', height=500, margin={"r":0,"t":30,"l":0,"b":0}, geo=dict(showframe=False))
fig.show()

# 3. go.Choropleth
### with GDP data

In [ ]:
# this just another exercise with country code data
# we want to try Choropleth feature now 

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
# df.tail()

In [ ]:
# this is the Choropleth plot with gdp data
# source: https://plotly.com/python/choropleth-maps/#world-choropleth-map

fig = go.Figure(data=go.Choropleth(
    locations = df['CODE'],
    z = df['GDP (BILLIONS)'],
    text = df['COUNTRY'],
    colorscale = 'Blues',
#     autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '$',
    colorbar_title = 'GDP<br>Billions US$',
))

fig.update_layout(
    title_text='2014 Global GDP',
    height=500,
    margin={"r":0,"t":50,"l":0,"b":0},
    geo=dict(
        showframe=False,
#         showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.5,
        y=0,
        xref='paper',
        yref='paper',
        text='Source:<a href="https://www.cia.gov/library/publications/download">\
            CIA World Factbook</a>',
        showarrow = False
    )]
)

fig.show()

# 4. go.Choropleth
### with covid19 data

In [23]:
# we use the same dataframe as in go.scattergeo

In [24]:
# this is a similar cell as the one in go.scattergeo
# Creating new dict for label dataframe 
label = {'Country': country_df['country'],
        'Confirmed': country_df['confirmed'],
        'Death': country_df['deaths'],
         'Recovered' : country_df['recovered']
        }

label_df = pd.DataFrame (label, columns = ['Country','Confirmed','Death','Recovered'])
label_df.head()

# removing .0 in str format
for col in label_df.columns:
    label_df[col] = label_df[col].astype(str).replace('\.0', '', regex=True)

# creating new column in label_df (called 'text') with the template for the text in choropleth 
label_df['text'] = '<b>' + label_df['Country'] + '</b>' + '<br>' + \
    'Confirmed : ' + label_df['Confirmed'].astype(str) + '<br>' + \
    'Death : ' + label_df['Death'].astype(str) + '<br>' + \
    'Recovered : ' + label_df['Recovered'].astype(str)

In [25]:
# this is the corona data plot with Choropleth feature.
# more color check https://community.plotly.com/t/plotly-colours-list/11730/3
# more color scale https://plotly.com/python/builtin-colorscales/

fig = go.Figure(data=go.Choropleth(
    locations = country_df['iso'],
    z = np.log(country_df['confirmed']),
    text = label_df['text'],
    marker_line_color='darkgray',
    colorscale = 'viridis',
    hoverinfo = ['all'],
    hovertemplate=
    "%{text}<br>"
    "<extra><b>%{location}</b></extra>",
    colorbar=dict(
        title="Number of Cases",
        tickvals=np.linspace(0,np.log(country_df['confirmed'].max()),7).astype(int),
        ticktext=np.append(0, np.exp(np.round(np.linspace(2,np.log(country_df['confirmed'].max()),6))).astype(int)),
    ),
))


fig.update_geos(projection_type="orthographic")
fig.update_layout(title = 'Confirmed Cases Globally', height=500, margin={"r":0,"t":30,"l":0,"b":0},
                     geo=dict(
                         showocean=True, oceancolor='lightcyan',
                         showlakes=True, lakecolor="Blue",
                         showrivers=True, rivercolor="Blue",
                         showframe=False
                     ),
)
fig.show()

# 5. go.Scatter
### with covid19 data

In [28]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [32]:
confirmed_df.columns = map(str.lower, confirmed_df.columns)
confirmed_df = confirmed_df.rename(columns={'country/region': 'country', 'province/state': 'state'})

death_df.columns = map(str.lower, death_df.columns)
death_df = death_df.rename(columns={'country/region': 'country', 'province/state': 'state'})

recovered_df.columns = map(str.lower, recovered_df.columns)
recovered_df = recovered_df.rename(columns={'country/region': 'country', 'province/state': 'state'})

In [33]:
death_df.head()

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,7,7,11,14,14,15,15,18,18,21
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,20,20,21,22,22,23,23,23,23,23
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,130,152,173,193,205,235,256,275,293,313
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,17,18,21,22,23,25,26,26,29,29
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [34]:
confirmed_clean  = confirmed_df.drop(['state', 'lat', 'long'], axis=1)
confirmed_clean.set_index('country', inplace=True)

death_clean  = death_df.drop(['state', 'lat', 'long'], axis=1)
death_clean.set_index('country', inplace=True)

recovered_clean  = recovered_df.drop(['state', 'lat', 'long'], axis=1)
recovered_clean.set_index('country', inplace=True)

# clean_new

In [35]:
fig = go.Figure(data=
                [go.Scatter(x = confirmed_clean.columns, y = confirmed_clean.loc['Indonesia'], name="confirmed"),
                go.Scatter(x = death_clean.columns, y = death_clean.loc['Indonesia'], name="death"),
                go.Scatter(x = recovered_clean.columns, y = recovered_clean.loc['Indonesia'], name="recovered")]
               )

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0.5,
        dtick = 8
    )
)

fig.update_xaxes(rangeslider_visible=True),
fig.update_yaxes(type="linear")

fig.show()